<h1> Linear models, matrix calculus, and gradient descent </h1>

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Let us understand machine learning through one of the simplest models possible, the linear regression model. Despite its simplicity, it is one of the most used models in machine learning (even modern ML). It is often used as a component in neural networks or on its own. 

The optimisation problem associated with this model is known as least squares. We are essentially trying to map a feature vector to some value by multiplying the features by learnable weights, and then summing the result. For instance, we might be interested in predicting home prices, and the features may be size of home, average price in area, and distance from the nearest city centre. One might find the weights to be [0,1,0] if the average price in the area is exactly the price of the home in our dataset. 

Mathematically, we are given a data matrix $A \in \mathbb{R}^{m \times n}$ and a vector of outcomes $y \in \mathbb{R}^m$, and attempt to
find a parameter vector $x \in \mathbb{R}^n$ which minimizes the residual $ \left\lVert A x - y \right\rVert_2^2 $. This finds what weight vector $x$ we can multiply with the data matrix $A$ to make the result as close to the prediction target as possible.

<h2> Solving the least-squares problem with matrix calculus </h2>

There are a lot of ways to solve this problem. In this notebook, we will consider two ways. The first is calculating the closed-form solution through matrix calculus. You can decompose it into sums, but it is quicker and cleaner to stay at the matrix level. 

To solve it, we first need to understand the concepts of a gradient and a jacobian. The gradient is equivalent to the first derivative in the multivariate world, and the jacobian is equivalent to the second derivative. Here are the definitions:

**Definition 1: Gradient**

Let $ f : \mathbb{R}^n \to \mathbb{R} $ be a differentiable function. The *gradient* of $ f $ is the function $ \nabla f : \mathbb{R}^n \to \mathbb{R}^n $ defined by
$$
\nabla f(\vec{x}) = 
\begin{bmatrix}
\frac{\partial f}{\partial x_1} (\vec{x}) \\
\vdots \\
\frac{\partial f}{\partial x_n} (\vec{x})
\end{bmatrix}
$$

**Definition 2: Jacobian**

Let $\vec{f} : \mathbb{R}^n \to \mathbb{R}^m$ be a differentiable function. The Jacobian of $ \vec{f} $ is the function $ D\vec{f} : \mathbb{R}^n \to \mathbb{R}^{m \times n} $ defined as:

$$
D\vec{f}(\vec{x}) = 
\begin{bmatrix}
\nabla f_1(\vec{x})^\top \\
\vdots \\
\nabla f_m(\vec{x})^\top
\end{bmatrix}
=
\begin{bmatrix}
\frac{\partial f_1}{\partial x_1}(\vec{x}) & \cdots & \frac{\partial f_1}{\partial x_n}(\vec{x}) \\
\vdots & \ddots & \vdots \\
\frac{\partial f_m}{\partial x_1}(\vec{x}) & \cdots & \frac{\partial f_m}{\partial x_n}(\vec{x})
\end{bmatrix}
$$

The jacobian of the gradient of a function is known as the Hessian of the function.

<h3> Using the gradient and jacobian to solve least-squares </h3>

As you can see, the gradient is a vector analogue to the classic univariate first derivative. The vector is defined where a single value (scalar) output depends on a vector of parameters. In machine learning, this is relevant when we have computed a loss value from predictions created by the parameters of a model. With least-squares, this is exactly what we are dealing with. The loss is calculated as $ \left\lVert A x - y \right\rVert_2^2 $ (note that the L2 norm gives a scalar output), and we used the parameters $ x $ to calculate the predictions. 

How do we use this to find the optimal parameters? We need to find the parameters that minimise the loss function. Further, the technique is completely analoguous to what is done in the univariate case (from math R1 and R2). For the univariate case, we set the derivative with respect to the parameters to 0, and calculate the parameter value that solves the equation. For the multivariate case, we set the gradient to the 0 vector, that is the vector with each entry equal to 0. Instead of a single equation, we get one equation for each of the $n$ parameters. These we can solve. 

However, how do we know that the parameters that lead to a zero gradient are the ones that minimise the loss? Again, the answer is the same as in the univariate case. We need to prove that the loss function is convex with respect to the parameters. Remember that a univariate convex function is defined by the second derivative being non-negative (bending upwards). How does this translate to the multivariate case? The answer is that the hessian needs to be positive-semidefinite (which is basically the non-negative in matrix world). 

A matrix $\mathbf{A}$ is positive semidefinite if it is symmetric and for any vector $\mathbf{x}, \mathbf{x}^T \mathbf{A} \mathbf{x} \geq 0$. This is equivalent to saying all eigenvalues must be non-negative (don't worry if you haven't learnt about this yet). You can get some intuition about PSD-matrices by considering diagonal matrices. That is, a matrix where all non-diagonal elements are equal to 0. In this case, the eigenvalues are the elements on the diagonal, and an all non-negative diagonal implies the matrix is PSD. We can interpet this is a "non-negative matrix". 

<h3> Finding the optimal parameters with matrix calculus </h3>

We can solve the least-squares problem in two steps:

1. Prove that the hessian of the loss function is positive-semidefinite ("the second derivative is non-negative")
2. Find the parameters that yield a gradient equal to 0

If you want, you can solve it yourself and then come back to the notebook. Otherwise, the solution is below.

We skip step 1 for now, but there are good solutions available on the web. We go straight to finding the parameters that minimise the loss, by setting the gradient equal to the 0 vector. 

The objective function for OLS is given by:

$$
f(x) = \left\lVert A x - y \right\rVert_2^2 = (A x - y)^T (A x - y)
$$

Expanding the quadratic form (using matrix algebra that you might not have seen in your courses yet), we have:

$$
f(x) = x^T A^T A x - 2 y^T A x + y^T y
$$

The gradient of $f(x)$ with respect to $x$ is:

$$
\nabla_x f(x) = 2 A^T A x - 2 A^T y
$$

To find the optimal $x$, set the gradient to zero and solve for $x$:

$$
2 A^T A x - 2 A^T y = 0
$$

Simplifying, we get:

$$
A^T A x = A^T y
$$

Assuming $A^T A$ is invertible, solve for $x$:

$$
x = (A^T A)^{-1} A^T y
$$

There we have it!

<h2> Solving the least-squares problem with gradient descent </h2>

An alternative solution method is through gradient descent. We introduced GD in the slides, but for a more comprehensive review, see https://ds100.org/fa23/lecture/lec13/, or https://people.eecs.berkeley.edu/~jrs/189/lec/03.pdf. Summarized:

$$
\mathbf{w}_{t+1} = \mathbf{w}_t - \eta \nabla J(\mathbf{w}_t)
$$

Now its time to apply these techniques to a real world dataset.

## 👨‍💻 Task 1: What is the relationship between hours studied and exam results?

Some might say that the more hous you log, the better your exam scores are. Let's find out how true that is.

In [16]:
data = pd.read_csv('regression_data/mission1.csv')

In [17]:
# TODO: Implement Linear Regression from scratch
class LinearRegression:
    def __init__(self, fit_method="closed_form"):
        
        if fit_method not in ["closed_form", "gradient_descent"]:
            raise ValueError("fit_method must be either 'closed_form' or 'gradient_descent'")

        self.fit_method = fit_method

    def fit(self, X, y):

        if self.fit_method == "closed_form":
            # Implement closed form solution
            ...
        elif self.fit_method == "gradient_descent":
            # Implement gradient descent solution
            ...

    def predict(self, X):
        pass


In [18]:
# TODO: Visualize the data
# TODO: Implement Linear Regression
# TODO: Estimate the coefficients (effect of studying) through Linear Regression
# TODO: Visualize the regression line


## 🧠 Task 2: Logistic Regression for Binary Classification

### Mission
1. From your Linear Regression model, make a Logistic Regression model that performs binary classification
2. Look into how you can use feature engineering to improve the accuracy
3. Solve it using a Decision Tree instead, and compare the results

**Performance goal**: 0.88 accuracy on the test set

In [19]:
data = pd.read_csv('regression_data/mission2.csv')

In [20]:
# TODO: Implement Logistic Regression from scratch
class LogisticRegression:
    def __init__(self):
        pass

    def fit(self, X, y):
        pass

    def predict(self, X):
        pass

In [21]:
# TODO: Visualize the data. Is it linearly separable?
# TODO: What can be done to make it linearly separable?